# Destinations des bateaux chargés de sel partis de la région

Point de départ : commentaire de Silvia ->

> je me suis avancée au pif, on peut vérifier aussi en juin, mais ça serait sympa de vérifier les destinations des navires chargés de sel en 1789 partis de notre région.

Méthodologie :

* prendre les pointcalls In 1789 avec source_subset = 'Poitou_1789' (cela nous donne les destinations de tous les navires partis de la région)
* scanner 'commodity_standardized_fr', 'commodity_standardized2_fr', 'commodity_standardized3_fr', 'commodity_standardized4_fr' pour ne garder que les pointcalls dont au moins une valeur est "Sel"
* agréger par nombre de pointcalls et tonnage cumulés

In [ ]:
from poitousprint import Portic

client = Portic()

In [ ]:
pointcalls_out = client.get_pointcalls(
    year = 1789, 
    source_subset = 'Poitou_1789',
    pointcall_action = 'In'
)

In [ ]:
commodity_fields = ['commodity_standardized_fr', 'commodity_standardized2_fr', 'commodity_standardized3_fr', 'commodity_standardized4_fr']

destinations = {}

for pc in pointcalls_out:
    for field in commodity_fields:
        if pc[field] is not None and pc[field].lower() == 'sel':
            port = pc["toponyme_fr"]
            tonnage = int(pc["tonnage"] if pc["tonnage"] is not None else 0)
            if port in destinations:
                destinations[port]["count"] += 1
                destinations[port]["tonnages_cumulated"] += tonnage
            else :
                destinations[port] = {
                    "count": 1,
                    "tonnages_cumulated": tonnage,
                    "state": pc["state_1789_fr"],
                    "in_region": True if pc["pointcall_admiralty"] in ["Marennes", "La Rochelle", "Sables d'Olonne"] else False
                }

destinations = [{"port": port, **data} for port, data in destinations.items()]

In [ ]:
from vega import VegaLite
import pandas as pd

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de pointcalls In par destination pour les bateaux transportant du sel (1789)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "count",
            "type": "quantitative", 
            "title": "nombre de pointcalls",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de pointcalls In par destination pour les bateaux transportant du sel (1789)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "tonnages_cumulated",
            "type": "quantitative", 
            "title": "tonnage cumulé",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les destinations dans la région (nombre de pointcalls)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "count",
            "type": "quantitative", 
            "title": "nombre de pointcalls",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == True]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les destinations dans la région (tonnage cumulé)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "tonnages_cumulated",
            "type": "quantitative", 
            "title": "tonnage cumulé",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == True]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les destinations hors la région (nombre de pointcalls)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "count",
            "type": "quantitative", 
            "title": "nombre de pointcalls",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == False]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les destinations hors la région (tonnages cumulés)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "tonnages_cumulated",
            "type": "quantitative", 
            "title": "tonnage cumulé",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "port",
            "title": "Port",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == False]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les états de destinations hors France (nombre de pointcalls)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "count",
            "type": "quantitative", 
            "aggregate": "sum",
            "title": "nombre de pointcalls",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "state",
            "title": "État",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == False and pc["state"] != 'France']))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Zoom sur les états de destinations hors France (tonnages cumulés)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "tonnages_cumulated",
            "type": "quantitative", 
            "aggregate": "sum",
            "title": "tonnage cumulé",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "type": "nominal",
            "field": "state",
            "title": "État",
            "sort": "-x"
        }
    }
}, pd.DataFrame([pc for pc in destinations if pc["in_region"] == False and pc["state"] != 'France']))